In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np

In [17]:

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


daraz_url = 'https://www.daraz.pk/'
driver.get(daraz_url)
driver.maximize_window()
time.sleep(5)


#THIS HOVERS OVER ELECTRONIC DEVICES SO IT SHOWS SMARTPHONE TAB ON WHICH I CAN CLICK
action = ActionChains(driver)
Electronic_Devices=driver.find_element(By.XPATH,'//*[@id="Level_1_Category_No1"]')
action.move_to_element(Electronic_Devices).perform()
time.sleep(2)

Smartphone=driver.find_element(By.XPATH,'//*[@id="J_5022174600"]/div/ul/ul[7]/li[1]')
Smartphone.click()
time.sleep(5)


In [18]:
def get_phone_data(phone_page_url):
    time.sleep(3)
    driver.execute_script("window.scrollTo(1000, 2000);")    
    check=driver.find_element(By.XPATH,'//*[@id="J_breadcrumb"]/li[2]/span/a/span').text.split('(')[0].strip()
    if check == 'Like New Phones' or check == 'Mobiles' or check=='Feature Phones':
        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center', behavior: 'smooth'});", driver.find_element(By.ID, 'module_product_detail'))
        time.sleep(3)  
        title = get_element_text(By.ID, 'module_product_title_1')
        price = get_element_text(By.XPATH, '//*[@id="module_product_price_1"]/div/div/span')
        rating = get_element_text(By.XPATH, '//*[@id="module_product_review"]/div/div/div[2]/div[1]/div[1]/div[1]/span')
        free_shipping = get_element_text(By.XPATH, '//*[@id="module_seller_delivery"]/div/div/div[3]/div/div[1]/div/div[1]/div[1]/div[1]/span[1]')
        seller_rating = get_element_text(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[1]/div[2]')
        ship_on_time = get_element_text(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]')
        
        try:
            specifications=driver.find_elements(By.XPATH, '//*[@id="module_product_detail"]/div/div[1]/div')
            spec=[]
            for i in specifications:
                spec.append(i.text)
        except:
            specifications=None

        driver.execute_script("window.scrollTo(2000,1000);")
        time.sleep(5)
        check=True
        try:
            next_key = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ant-pagination-next')))
        except TimeoutException:
            check=False
        reviews = []
        for i in range(1,4):
            for j in range(1, 4):
                try:
                    review_xpath = f'//*[@id="module_product_review"]/div/div/div[3]/div[{j}]/div[2]'
                    review_element = driver.find_element(By.XPATH,review_xpath)
                    review_text = review_element.text
                    reviews.append(review_text)
                except:
                    reviews.append('N/A')
            if(check==True):
                if "disabled" in next_key.get_attribute("class"):
                    break
                next_key.click()
                time.sleep(1)
        driver.back()
        phones_reviews.append(reviews)

        return {
            'ID':count,
            'Product Title': title,
            'Price': price,
            'Rating': rating,
            'Free Shipping Status': free_shipping,
            'Seller Rating': seller_rating,
            'Ship on Time': ship_on_time
            'Product Link': phone_page_url,
            'Specifications':spec,
        }
    else:
        driver.back()

def get_element_text(by, selector):
    try:
        element = driver.find_element(by, selector)
        return element.text.split('(')[0].strip() if element else None
    except:
        return 0

phones_data = []
page_number = 1
phones_reviews =[]

while page_number <= 5:
    print(f"Extracting data from page {page_number}")

    phone_links = driver.find_elements(By.ID, 'id-a-link')
    phone_links_hrefs = [link.get_attribute('href') for link in phone_links]
    visited_links = set()

    for count,link in enumerate(phone_links_hrefs):
        if(count%4==0):
            driver.execute_script("window.scrollTo(0, window.scrollY + 200);")
        if link not in visited_links:
            driver.get(link) 
            phone_info = get_phone_data(driver.current_url,count+1)

            if phone_info:
                phones_data.append(phone_info)

            visited_links.add(link) 

    print(f"Number of phone data after collecting from page {page_number}: {len(phones_data)}")

    next_page_button = driver.find_element(By.XPATH,'//*[@id="root"]/div/div[4]/div[1]/div/div[1]/div[3]/div/div/ul/li[9]/a')
    next_page_button.click()
    time.sleep(5)
    page_number += 1

driver.quit()

phones_df = pd.DataFrame(phones_data)
reviews_df= pd.DataFrame(phones_reviews)

Extracting data from page 1
Number of phone data after collecting from page 1: 40
Extracting data from page 2
Number of phone data after collecting from page 2: 80
Extracting data from page 3
Number of phone data after collecting from page 3: 120
Extracting data from page 4
Number of phone data after collecting from page 4: 160
Extracting data from page 5
Number of phone data after collecting from page 5: 200


In [19]:
phones_df

,ID,Product Title,Price,Rating,Free Shipping Status,Seller Rating,Ship on Time,Product Link,Specifications
0,1,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,"Rs. 34,999",4.4,Free Delivery,New Seller,100%,https://www.daraz.pk/products/samsung-a14-4gb6...,[• Samsung A14 with 4GB/64GB memory.• 5000 mAh...
1,3,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,"Rs. 49,999",4.7,Standard Delivery,92%,100%,https://www.daraz.pk/products/redmi-note-12-8-...,[Redmi Note 12 8GB RAM + 128GB ROM PTA Approve...
2,5,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",4.7,Free Delivery,New Seller,100%,https://www.daraz.pk/products/infinix-note-30-...,[Release Date2023-05-22SIM SupportDual SIM (Na...
3,7,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,"Rs. 19,399",4.7,Standard Delivery,92%,100%,https://www.daraz.pk/products/redmi-a2-3gb64gb...,[Display: 6.52” HD+ Dot Drop displayResolution...
4,9,Realme Narzo 50A Prime,"Rs. 32,999",4.2,Free Delivery,New Seller,100%,https://www.daraz.pk/products/realme-narzo-50a...,[ColourXanh Tia Chop = Lightning Blueđen = Bla...
...,...,...,...,...,...,...,...,...,...
195,71,Tecno SPARK 10 PRO 8GB RAM - 128GB ROM 50 MP C...,"Rs. 39,499",5.0,Standard Delivery,88%,100%,https://www.daraz.pk/products/10-8gb-128gb-50-...,[Starry Black/Pearl White/Magic Skin(Black and...
196,73,Redmi Note 12,"Rs. 50,999",4.5,Standard Delivery,93%,100%,https://www.daraz.pk/products/12-8gb-128gb-pta...,[CPU ------------- Snapdragon® 685 Octa-core C...
197,75,Infinix Smart 7 HD - 2GB RAM 64GB ROM - 6.6 In...,"Rs. 21,499",4.0,Free Delivery,89%,100%,https://www.daraz.pk/products/7-hd-2gb-64gb-66...,[PTA Approved\nIPS LCD\n6.6 Inches LCD\n2GB RA...
198,77,Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...,"Rs. 29,999",4.9,Free Delivery,91%,74%,https://www.daraz.pk/products/tecno-spark-10c-...,[General FeaturesRelease Date2023-06-16SIM Sup...


In [20]:
phones_df.to_csv('phones.csv', index=False)

In [36]:
reviews_df.to_csv('reviews.csv', index=False)

In [46]:
#CHATBOT
import pandas as pd
df=pd.read_csv('phones.csv')

In [47]:
df['Price'] = df['Price'].str.replace('Rs.', '').str.replace(',', '').astype(float)
df['Price']

In [4]:
df[df['Product Title'].str.contains("Samsung A14",case=False)]

,ID,Product Title,Price,Rating,Free Shipping Status,Seller Rating,Ship on Time,Product Link,Specifications
0,1,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,34999.0,4.4,Free Delivery,New Seller,100%,https://www.daraz.pk/products/samsung-a14-4gb6...,"[""• Samsung A14 with 4GB/64GB memory.• 5000 mA..."
179,39,Samsung A14 - 6GB RAM - 128GB ROM - 5000mAh Ba...,56499.0,5.0,Free Delivery,89%,100%,https://www.daraz.pk/products/6gb-a14-128gb-50...,['PTA Approved\n6GB RAM\n128GB ROM\n5000mAh Ba...


In [48]:
def extract_intent(query):
    words = word_tokenize(query)
    filtered_words = [word.lower() for word in words]
    
    if 'hello' in filtered_words or 'hi' in filtered_words or 'hey' in filtered_words:
        return 'greet'
    elif 'top' in filtered_words and 'phone' in filtered_words:
        if 'brand' in filtered_words:
            return 'top_phones_brand'
        return 'top_phones'
    elif 'between' in filtered_words and 'price' in filtered_words:
        return 'between_price'
    elif 'price' in filtered_words and 'rating' in filtered_words:
        if 'above' in filtered_words or 'over' in filtered_words:
            return 'above_price_rating'
        elif 'below' in filtered_words or 'under' in filtered_words:
            return 'below_price_rating'
    elif ('under' in filtered_words or 'below' in filtered_words) and 'price' in filtered_words:
        return 'under_price'
    elif ('over' in filtered_words or 'above' in filtered_words) and 'price' in filtered_words:
        return 'over_price'
    elif 'specific' in filtered_words and 'phone' in filtered_words:
        return 'specific_phone'
    elif 'cheapest' in filtered_words and 'phone' in filtered_words:
        if 'specs' in filtered_words:
            return 'specs_cheapest_phone'
        return 'cheapest_phone'
    elif 'expensive' in filtered_words and 'phone' in filtered_words:
        if 'specs' in filtered_words:
            return 'specs_expensive_phone'
        return 'expensive_phone'
    elif 'specs' in filtered_words and 'phone' in filtered_words:
        if 'best' in filtered_words:
            return 'best_specs'
        else:
            return 'spec_phone'
    elif 'number' in filtered_words and 'phones' in filtered_words and 'brand' in filtered_words:
        return 'phones_per_brand'

    elif 'names' in filtered_words and 'brand' in filtered_words:
        if 'every' in filtered_words:
            return 'brand_names'
    elif 'above' in filtered_words and 'rating' in filtered_words:
        return 'above_rating'
        
    elif 'below' in filtered_words and 'rating' in filtered_words:
        return 'below_rating'

    return None  

def handle_query(intent, query):
    if intent=='greet':
        return "HI!! HOW CAN I HELP YOU TODAY!!!!"
    
    elif intent == 'between_price':
        # Extract minimum and maximum prices from the query
        prices = [int(word) for word in word_tokenize(query) if word.isdigit()]
        if len(prices) == 2:
            min_price, max_price = min(prices), max(prices)
            phones_between_prices = df[(df['Price'] >= min_price) & (df['Price'] <= max_price)]
            return phones_between_prices[['Product Title', 'Price']]
        
    elif intent=="top_phones":
        number = [int(word) for word in word_tokenize(query) if word.isdigit()]
        Top_Rating = df.sort_values(by='Rating', ascending=False)
        return Top_Rating.head(number[0])
    elif intent=="top_phones_brand":
        number = [int(word) for word in word_tokenize(query) if word.isdigit()]
        words = query.split()
        brand_index = words.index('brand')
        brand = words[brand_index - 1]
        filtered_rows = df[df['Product Title'].str.contains(brand)]
        Top_Rating = filtered_rows.sort_values(by='Rating', ascending=False)
        return Top_Rating.head(number[0])
        
    elif intent == 'under_price':
        # Extract the maximum price from the query
        max_price = max([int(word) for word in word_tokenize(query) if word.isdigit()])
        phones_under_price = df[df['Price'] < max_price]
        return phones_under_price[['Product Title', 'Price']]

    elif intent == 'over_price':
        # Extract the minimum price from the query
        min_price = min([int(word) for word in word_tokenize(query) if word.isdigit()])
        phones_over_price = df[df['Price'] > min_price]
        return phones_over_price[['Product Title', 'Price']]
    
    elif intent == 'spec_phone':
        specs = query.split(':')[-1].strip()
        phone_with_specs =df[df['Specifications'].str.contains(specs)]
        if(phone_with_specs.empty):
            return "Couldnt locate in the dataframe!"
        return phone_with_specs
    
    elif intent=='specs_cheapest_phone':
        cheapest_phone = df[df['Price'] == df['Price'].min()]
        return cheapest_phone.iloc[0]['Specifications']
     
    elif intent=='specs_expensive_phone':
        most_expensive_phone = df[df['Price'] == df['Price'].max()]
        return most_expensive_phone.iloc[0]['Specifications']
    
    elif intent == 'cheapest_phone':
        cheapest_phone = df[df['Price'] == df['Price'].min()]
        if not cheapest_phone.empty:
            phone_name = cheapest_phone.iloc[0]['Product Title']
            phone_price = cheapest_phone.iloc[0]['Price']
            phone_link = cheapest_phone.iloc[0]['Product Link']
            return f"The cheapest phone is {phone_name} priced at Rs. {phone_price}. Product link: {phone_link}"

    elif intent == 'expensive_phone':
        most_expensive_phone = df[df['Price'] == df['Price'].max()]
        if not most_expensive_phone.empty:
            phone_name = most_expensive_phone.iloc[0]['Product Title']
            phone_price = most_expensive_phone.iloc[0]['Price']
            phone_link = most_expensive_phone.iloc[0]['Product Link']
            return f"The most expensive phone is {phone_name} priced at Rs. {phone_price}. Product link: {phone_link}"

    elif intent == 'specific_phone':
        # Extract the specific phone name from the query
        query_words = query.split()
        if len(query_words) >= 3:
            phone_name = ' '.join(query_words[2:])  # Extract the model name (all words after the brand name)

            specific_phone = df[df['Product Title'].str.contains(phone_name, case=False)]
                
            if not specific_phone.empty:
                return specific_phone
            else:
                return f"Sorry, {query} is not found in the database."
        else:
            return "Please provide both brand and model for this query."
    
    elif intent == 'phones_per_brand':
        brand_counts = df['Product Title'].apply(lambda x: x.split()[0]).value_counts()
        return brand_counts.to_dict() if not brand_counts.empty else "No data available for phone count per brand."
    
    elif intent == 'brand_names':
        brand_names = df['Product Title'].apply(lambda x: x.split()[0]).unique()
        return brand_names.tolist() if brand_names.any() else "No brand names found in the database."
    elif intent == 'above_rating':
        rating = int(query.split(':')[-1].strip())
        phones_above_rating = df[df['Rating'] > rating]
        return phones_above_rating[['Product Title', 'Rating']]

    elif intent == 'below_rating':
        rating = int(query.split(':')[-1].strip())
        phones_below_rating = df[df['Rating'] < rating]
        return phones_below_rating[['Product Title', 'Rating']]

    elif intent == 'above_price_rating':
        price, rating = None, None
        words = query.split()
        for i, word in enumerate(words):
            if word.isdigit():
                if price is None:
                    price = int(word)
                else:
                    rating = int(word)
                    break

        if price and rating:
            phones_above_price_rating = df[(df['Price'] > price) & (df['Rating'] > rating)]
            return phones_above_price_rating[['Product Title', 'Price', 'Rating']]
        
    elif intent == 'below_price_rating':
        price, rating = None, None
        words = query.split()
        for i, word in enumerate(words):
            if word.isdigit():
                if price is None:
                    price = int(word)
                else:
                    rating = int(word)
                    break

        if price and rating:
            phones_above_price_rating = df[(df['Price'] < price) & (df['Rating'] < rating)]
            return phones_above_price_rating[['Product Title', 'Price', 'Rating']]

    else:
        return "Sorry, I couldn't understand the query."

In [49]:
import panel as pn
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [50]:
stop_words = set(stopwords.words('english'))

In [51]:
import pandas as pd
import panel as pn

# Initialize counter for number of questions asked
num_questions_asked = 0

# Function to calculate average price
def calculate_average_price():
    return df['Price'].mean()

# Function to calculate average rating
def calculate_average_rating():
    return df['Rating'].mean()
    
# Initialize conversation history variable
conversation_history = []

def query_response(user_query):
    intent=extract_intent(user_query)
    return handle_query(intent,user_query)

# Function to update statistics based on user queries and save conversation history
def update_statistics(event):
    global num_questions_asked
    global conversation_history
    
    num_questions_asked += 1
    
    user_query = query_input.value
    response = query_response(user_query)
    
    # Create Markdown panes for user query and bot response
    user_query_pane = pn.pane.Markdown(f"User: {user_query}")
    
    if isinstance(response, pd.DataFrame):
        # Display DataFrame
        response_widget = pn.widgets.DataFrame(response)
    else:
        # Display text
        response_widget = pn.pane.Markdown(f"Chatbot: {response}")
    
    spacer_user_response = pn.Spacer(height=5)  # Spacer between user query and chatbot response
    spacer_query = pn.Spacer(height=10)  # Spacer between subsequent queries
    
    # Append conversation history
    conversation_history.append(f"User: {user_query}")
    conversation_history.append(f"Chatbot: {response}")
    
    # Clear previous content in the conversation panel
    conversation_panel.clear()
    
    conversation_panel.append(user_query_pane)
    conversation_panel.append(spacer_user_response)
    conversation_panel.append(response_widget)
    
    # Update displayed statistics
    num_questions_pane.object = f"<h3>Questions Asked: {num_questions_asked}</h3>"

    # Save conversation history to a file
    save_conversation_history()
    
# Function to save conversation history to a file
def save_conversation_history():
    with open("conversation_history.txt", "w") as file:
        file.write("\n".join(conversation_history))


conversation_panel = pn.Column(sizing_mode='stretch_width', align='center')


query_input = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
query_button = pn.widgets.Button(name="Query", button_type='primary')
query_button.on_click(update_statistics)

conversation_panel = pn.Column(sizing_mode='stretch_width', align='center')


avg_price = calculate_average_price()
avg_rating = calculate_average_rating()



avg_price_pane = pn.pane.HTML(f"<h3>Average Product Price: Rs. {avg_price:.2f}</h3>", width=300, style={'background': '#f9f9f9', 'padding': '10px', 'border': '1px solid #ccc', 'border-radius': '5px'})
avg_rating_pane = pn.pane.HTML(f"<h3>Average Product Rating: {avg_rating:.2f}</h3>", width=300, style={'background': '#f9f9f9', 'padding': '10px', 'border': '1px solid #ccc', 'border-radius': '5px'})
num_questions_pane = pn.pane.HTML(f"<h3>Questions Asked: {num_questions_asked}</h3>", width=300, style={'background': '#f9f9f9', 'padding': '10px', 'border': '1px solid #ccc', 'border-radius': '5px'})


user_query_column = pn.Column(conversation_panel)


statistics_column = pn.Column(avg_price_pane, avg_rating_pane, num_questions_pane, align='center')


input_panel = pn.Column(pn.Spacer(height=50), query_input, query_button, align='center')


grid = pn.GridSpec(sizing_mode='stretch_both')
grid[0, 0] = user_query_column
grid[0, 1] = input_panel
grid[0, 2] = statistics_column

grid.show()
